<a href="https://colab.research.google.com/github/amalvarezme/AnaliticaDatos/blob/master/UNAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis impacto Decreto 1279 - UNAL



In [ ]:
#Lectura datos
FILEID = "1zgDdYXuqGJXAPTw3QJUMUvpFWpo-HOgZ"

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O costo_personal_unal.xlsx && rm -rf /tmp/cookies.txt


FILEID = "1X5gy9CmyUmB5c6l5VUwmGx4Q-AcAxtos"

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O historico_puntos_unal.xlsx && rm -rf /tmp/cookies.txt

!dir

In [ ]:
!pip install pandas openpyxl

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Define the folder path
path_ = "resultadosUNAL"

# Create the folder
try:
    os.makedirs(path_, exist_ok=True)  # `exist_ok=True` avoids error if the folder already exists
    print(f"Folder '{path_}' created successfully!")
except Exception as e:
    print(f"Error creating folder: {e}")

#ocasionales
file_path = 'costo_personal_unal.xlsx'
start_row = 2  # 0-indexed, this means row 6 in Excel
end_row = 13   #
use_columns = ['Año', 'Total general']  # Columns by Excel labels

costos_ocasionales_UN = pd.read_excel(
    file_path,
    usecols=use_columns,       # Specify columns to read
    skiprows=start_row,        # Skip rows before start_row
    nrows=end_row - start_row,  # Number of rows to read
    sheet_name='Hoja2'
)

costos_ocasionales_UN.head()

#planta recursos nacion
file_path = 'costo_personal_unal.xlsx'
start_row = 20  # 0-indexed, this means row 6 in Excel
end_row = 31   #
use_columns = ['Año', 'Gastos de Personal RN', 'Gastos de Personal - Docentes', '% Participación linea Docentes']  # Columns by Excel labels

costos_docentes_nacion_UN = pd.read_excel(
    file_path,
    usecols=use_columns,       # Specify columns to read
    skiprows=start_row,        # Skip rows before start_row
    nrows=end_row - start_row,  # Number of rows to read
    sheet_name='Hoja2'
)

costos_ocasionales_UN.head(20)

# Costo ocasionales

In [ ]:
from matplotlib.ticker import FuncFormatter

# Calculate percentage increase between years
costos_ocasionales_UN['% Incremento Anual'] = costos_ocasionales_UN['Total general'].pct_change() * 100

# Formatter function for y-axis in thousands
def thousands_formatter(x, pos):
    return f'${int(x/1e6)}M'

# Plotting
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar plot for values
ax1.bar(costos_ocasionales_UN['Año'], costos_ocasionales_UN['Total general'], color='skyblue', label='Costo ocasionales',alpha=0.5)
ax1.set_xlabel('Año')
ax1.set_ylabel('Valor $', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_title('Costo Ocasionales UNAL 2013 - 2023')
ax1.set_xticks(costos_ocasionales_UN['Año'])
# Adjust y-axis ticks to thousands
ax1.yaxis.set_major_formatter(FuncFormatter(thousands_formatter))

# Create a second y-axis for percentage increase
ax2 = ax1.twinx()
ax2.plot(costos_ocasionales_UN['Año'], costos_ocasionales_UN['% Incremento Anual'], color='red', marker='o', label='% Incremento Anual')
ax2.set_ylabel('% Incremento Anual', color='red')
ax2.tick_params(axis='y', labelcolor='red')


# Calculate average percentage increase and standard deviation
avg_increase = costos_ocasionales_UN['% Incremento Anual'].mean()
med_increase = costos_ocasionales_UN['% Incremento Anual'].median()
std_increase = costos_ocasionales_UN['% Incremento Anual'].std()

# Add fill-between for ±2 std dev of % increase
ax2.plot(costos_ocasionales_UN['Año'],avg_increase*np.ones((costos_ocasionales_UN.shape[0],1)),'y--',label='Media Inc.: '+str(round(avg_increase,2))+'%')

ax2.plot(costos_ocasionales_UN['Año'],med_increase*np.ones((costos_ocasionales_UN.shape[0],1)),'g-.',label='Mediana Inc.: '+str(round(med_increase,2))+'%')


#ax2.fill_between(
 #   costos_ocasionales_UN['Año'],
  #  avg_increase - 1 * std_increase,
  #  avg_increase + 1 * std_increase,
  #  color='orange',
  #  alpha=0.3,
  #  label='± Std Dev'
#)


# Add legends
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9), bbox_transform=ax1.transAxes)

# Show plot
plt.tight_layout()
plt.grid()
plt.savefig(path_+'/costo_ocasionales_UNAL.png',dpi = 300)
plt.show()



# Gastos Personal con Recursos Nación

In [ ]:
costos_docentes_nacion_UN.columns

In [ ]:
#costos_docentes_nacion_UN


fig, ax1 = plt.subplots(figsize=(10, 6))

# Width of a bar
width = 0.4

# Plotting
ax1.bar(costos_docentes_nacion_UN['Año']-width/2, costos_docentes_nacion_UN['Gastos de Personal RN'], width, label='Gastos Personal RN',alpha=0.5)
ax1.bar(costos_docentes_nacion_UN['Año']+width/2, costos_docentes_nacion_UN['Gastos de Personal - Docentes'], width, label='Gastos de Personal - Docentes',alpha=0.5)
ax1.set_xlabel('Año')
ax1.set_ylabel('Valor $', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_title('% Gastos de Personal Recursos Nación (RN) UNAL 2013-2023')
ax1.set_xticks(costos_docentes_nacion_UN['Año'])
# Adjust y-axis ticks to thousands
ax1.yaxis.set_major_formatter(FuncFormatter(thousands_formatter))

# Create a second y-axis for percentage increase
# Calculate percentage increase between years
costos_docentes_nacion_UN['% Incremento Anual GP RN'] = costos_docentes_nacion_UN['Gastos de Personal RN'].pct_change() * 100

costos_docentes_nacion_UN['% Incremento Anual GP'] = costos_docentes_nacion_UN['Gastos de Personal - Docentes'].pct_change() * 100



ax2 = ax1.twinx()

#ax2.plot(costos_docentes_nacion_UN['Año'], costos_docentes_nacion_UN['% Incremento Anual GP RN'], color='red', marker='o', label='% Incremento Anual GP RN')
#ax2.set_ylabel('% Incremento Anual', color='red')
#ax2.tick_params(axis='y', labelcolor='red')

ax2.plot(costos_docentes_nacion_UN['Año'], costos_docentes_nacion_UN['% Incremento Anual GP'], color='red', marker='o', label='% Incremento Anual GP')
ax2.set_ylabel('% Incremento Anual GP', color='red')
ax2.tick_params(axis='y', labelcolor='red')


# Calculate average percentage increase and standard deviation
avg_increase = costos_docentes_nacion_UN['% Incremento Anual GP'].mean()
med_increase = costos_docentes_nacion_UN['% Incremento Anual GP'].median()
std_increase = costos_docentes_nacion_UN['% Incremento Anual GP'].std()

# Add fill-between for ±2 std dev of % increase
ax2.plot(costos_docentes_nacion_UN['Año'],avg_increase*np.ones((costos_docentes_nacion_UN.shape[0],1)),'y--',label='Media Inc.: '+str(round(avg_increase,2))+'%')

ax2.plot(costos_docentes_nacion_UN['Año'],med_increase*np.ones((costos_docentes_nacion_UN.shape[0],1)),'g-.',label='Mediana Inc.: '+str(round(med_increase,2))+'%')

# Add legends
fig.legend(loc='upper left', bbox_to_anchor=(0.005, 1.015), bbox_transform=ax1.transAxes)


plt.grid()
plt.savefig(path_+'/gastos_personal_nacion_UNAL.png',dpi = 300)
plt.show()


# Aumento Salario Puntos 1279

In [ ]:
file_path = 'historico_puntos_unal.xlsx'

start_row = 0  # 0-indexed, this means row 6 in Excel
end_row = 21   #
use_columns = ['AÑO ACTA', 'PUNTOS ASIGNADOS']  # Columns by Excel labels


puntos_asignados_UN = pd.read_excel(
    file_path,
    usecols=use_columns,       # Specify columns to read
    skiprows=start_row,        # Skip rows before start_row
    nrows=end_row - start_row,  # Number of rows to read
    sheet_name='10. PUNTOS PROD'
)



In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))


# Plotting
ax1.bar(puntos_asignados_UN['AÑO ACTA'], puntos_asignados_UN['PUNTOS ASIGNADOS'], label='PUNTOS ASIGNADOS UNAL',alpha=0.5)

ax1.set_xlabel('Año')
ax1.set_ylabel('#TOTAL PUNTOS ASIGNADOS', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_title('PUNTOS ASIGNADOS UNAL 2003-2023')
ax1.set_xticks(puntos_asignados_UN['AÑO ACTA'])  # Ensure correct positioning for labels
ax1.set_xticklabels(list(puntos_asignados_UN['AÑO ACTA']),rotation=45)  # Rotate labels by 45 degrees

# Create a second y-axis for percentage increase
# Calculate percentage increase between years
puntos_asignados_UN['% Incremento Anual Ptos. Asignados'] = puntos_asignados_UN['PUNTOS ASIGNADOS'].pct_change() * 100


ax2 = ax1.twinx()
ax2.plot(puntos_asignados_UN['AÑO ACTA'], puntos_asignados_UN['% Incremento Anual Ptos. Asignados'], color='red', marker='o', label='% Incremento Anual Ptos. Asignados')
ax2.set_ylabel('% Incremento Anual Ptos. Asignados', color='red')
ax2.tick_params(axis='y', labelcolor='red')


# Calculate average percentage increase and standard deviation
avg_increase = puntos_asignados_UN['% Incremento Anual Ptos. Asignados'].mean()
med_increase = puntos_asignados_UN['% Incremento Anual Ptos. Asignados'].median()
std_increase = puntos_asignados_UN['% Incremento Anual Ptos. Asignados'].std()

# Add fill-between for ±2 std dev of % increase
ax2.plot(puntos_asignados_UN['AÑO ACTA'],avg_increase*np.ones((puntos_asignados_UN.shape[0],1)),'y--',label='Media Inc.: '+str(round(avg_increase,2))+'%')

ax2.plot(puntos_asignados_UN['AÑO ACTA'],med_increase*np.ones((puntos_asignados_UN.shape[0],1)),'y--',label='Mediana Inc.: '+str(round(med_increase,2))+'%')

# Add legends
fig.legend(loc='upper left', bbox_to_anchor=(0.005, 1.015), bbox_transform=ax1.transAxes)


plt.grid()
plt.savefig(path_+'/puntos_asignados_UNAL.png',dpi = 300)
plt.show()


In [ ]:


# Plotting
fig, ax1 = plt.subplots(figsize=(10, 6))


# Plotting
ax1.bar(puntos_asignados_UN.iloc[10:,0], puntos_asignados_UN.iloc[10:,1], label='PUNTOS ASIGNADOS UNAL',alpha=0.5)

ax1.set_xlabel('Año')
ax1.set_ylabel('#TOTAL PUNTOS ASIGNADOS', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_title('PUNTOS ASIGNADOS UNAL 2013-2023')
ax1.set_xticks(puntos_asignados_UN.iloc[10:,0])  # Ensure correct positioning for labels
ax1.set_xticklabels(list(puntos_asignados_UN.iloc[10:,0]),rotation=45)  # Rotate labels by 45 degrees




ax2 = ax1.twinx()
ax2.plot(puntos_asignados_UN.iloc[10:,0], puntos_asignados_UN.iloc[10:,-1], color='red', marker='o', label='% Incremento Anual Ptos. Asignados')
ax2.set_ylabel('% Incremento Anual Ptos. Asignados', color='red')
ax2.tick_params(axis='y', labelcolor='red')


# Calculate average percentage increase and standard deviation
avg_increase = np.mean(puntos_asignados_UN.iloc[10:,-1])
med_increase = np.median(puntos_asignados_UN.iloc[10:,-1])


# Add fill-between for ±2 std dev of % increase
ax2.plot(puntos_asignados_UN.iloc[10:,0],avg_increase*np.ones((puntos_asignados_UN.iloc[10:,0].shape[0],1)),'r--',label='Media Inc.: '+str(round(avg_increase,2))+'%')

ax2.plot(puntos_asignados_UN.iloc[10:,0],med_increase*np.ones((puntos_asignados_UN.iloc[10:,0].shape[0],1)),'y--',label='Mediana Inc.: '+str(round(med_increase,2))+'%')

# Add legends
fig.legend(loc='upper right', bbox_to_anchor=(0.95, 0.975), bbox_transform=ax1.transAxes)


plt.grid()
plt.savefig(path_+'/puntos_asignados_UNAL_10.png',dpi = 300)
plt.show()

In [ ]:
file_path = 'historico_puntos_unal.xlsx'

start_row = 0  # 0-indexed, this means row 6 in Excel
end_row = 62   #
use_columns = ['SUELDO MIN', 'SUELDO MAX','2023_']  # Columns by Excel labels


sueldos2023_UN = pd.read_excel(
    file_path,
    usecols=use_columns,       # Specify columns to read
    skiprows=start_row,        # Skip rows before start_row
    nrows=end_row - start_row,  # Number of rows to read
    sheet_name='5. SUELDOS PLANTA'
)
print(sueldos2023_UN['2023_'].sum())

sueldos2023_UN.fillna(0,inplace=True) #fill NAN



In [ ]:
#range
for i in range(sueldos2023_UN.shape[0]):
   sueldos2023_UN.loc[[i],['RANGE']] = str((sueldos2023_UN.loc[[i],['SUELDO MIN']]/1e6).to_numpy(dtype=int)[0][0]) + '-' + str((sueldos2023_UN.loc[[i],['SUELDO MAX']]/1e6).to_numpy(dtype=int)[0][0])+ 'M'

sueldos2023_UN['RANGE']

In [ ]:
#medium salary
#percentage

sueldos2023_UN['SUELDO APROX.'] = (sueldos2023_UN['SUELDO MIN']+sueldos2023_UN['SUELDO MAX'])/2

sueldos2023_UN['PORCENTAGE PROFESORES'] = (sueldos2023_UN['2023_']/sueldos2023_UN['2023_'].sum())*100

sueldos2023_UN['PERCENTILES'] = np.cumsum(sueldos2023_UN['PORCENTAGE PROFESORES'])

Nprofes = sueldos2023_UN['2023_'].sum()
print(Nprofes)


In [ ]:
from scipy.spatial.distance import cdist

percentiles_ = np.array([25,50,75,99])

dis_ = cdist(sueldos2023_UN['PERCENTILES'].to_numpy().reshape(-1,1),percentiles_.reshape(-1,1))

ind_ = np.argmin(dis_,axis=0)

print(dis_.shape,ind_.shape)
sueldos2023_UN.loc[ind_,['PERCENTILES','SUELDO APROX.']]


In [ ]:
sueldos2023_UN.plot(x='SUELDO APROX.',y='PERCENTILES',marker='o')

In [ ]:
avg_salary = sueldos2023_UN['SUELDO APROX.'].to_numpy().dot(sueldos2023_UN['PORCENTAGE PROFESORES'].to_numpy()/100)

med_salary = sueldos2023_UN.loc[ind_[1],'SUELDO APROX.']

print(avg_salary, med_salary)

ind_avg = np.argmin(cdist(sueldos2023_UN['SUELDO APROX.'].to_numpy().reshape(-1,1),np.array([avg_salary]).reshape(-1,1)))



In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar plot for values
ax1.bar(
range(sueldos2023_UN.shape[0]),
sueldos2023_UN['PORCENTAGE PROFESORES'], color='blue', label='SUELDOS PLANTA UNAL 2023',alpha=0.5)
ax1.set_xlabel('RANGO')
ax1.set_ylabel('% PROFESORES DE PLANTA', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_title(f'SUELDOS PROFESORES DE PLANTA UNAL 2023\n TOTAL PROFESORES: {int(Nprofes)}')

#average median salary

ax1.plot(ind_avg*np.ones((20,1)),np.linspace(0,9.5,20),'r--',label='Media: $'+str(round(avg_salary/1e6,1))+'M',linewidth=3)

ax1.plot(ind_[1]*np.ones((20,1)),np.linspace(0,9.5,20),'y--',label='Mediana: $'+str(round(med_salary/1e6,1))+'M',linewidth=3)
ax1.set_xticks(range(sueldos2023_UN.shape[0]))  # Ensure correct positioning for labels
ax1.set_xticklabels(sueldos2023_UN['RANGE'],rotation=90)  #
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(path_+'/sueldos_2023_UNAL.png',dpi = 300)
plt.show()


# Distribución Salarios Docentes de Planta


In [ ]:
sueldos2023_UN.head()

In [ ]:
percentiles_ = np.array([20,40,60,80,99])

dis_ = cdist(sueldos2023_UN['PERCENTILES'].to_numpy().reshape(-1,1),percentiles_.reshape(-1,1))

ind_ = np.argmin(dis_,axis=0)
ind_ = np.append(ind_,sueldos2023_UN.shape[0]-1)
print(sueldos2023_UN.loc[ind_,['PERCENTILES','SUELDO APROX.']])

pi = -1
bar_sueldos = []
por_profes_q = []
for i in ind_:#percentile  25, 50, 75, 99
    xx = np.array([])
    print(pi+1,i)
    for j in range(pi+1,i+1):
        xx = np.append(xx,sueldos2023_UN.loc[j,'SUELDO APROX.']*np.ones((1,sueldos2023_UN.loc[j,'2023_'].astype(int)))[0])
    pi = i
    bar_sueldos.append(xx)
    por_profes_q.append(xx.shape[0]/Nprofes*100)
    print(por_profes_q[-1].round(1))

In [ ]:
# Define colors for each boxplot
colors = ['skyblue', 'lightgreen', 'lightcoral', 'gold', 'plum', 'lightsalmon']

# Create the figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Plot the boxplots
box = ax.boxplot(bar_sueldos, patch_artist=True, showmeans=True)

# Apply colors to each boxplot
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)


ax.set_xticks(range(1, 7))
ax.set_xticklabels(['0-20%', '20-40%', '40-60%', '60-80%','80-99%', 'TOP 1%'])

ax.set_xlabel('QUANTILES SALARIOS UNAL 2023')
ax.set_ylabel('Valor $')#, color='blue')
#ax.tick_params(axis='y', labelcolor='blue')
ax.set_title('DISTRIBUCIÓN POR QUANTILES SALARIOS PROFESORES DE PLANTA UNAL 2023')

# Adjust y-axis ticks to thousands
ax.yaxis.set_major_formatter(FuncFormatter(thousands_formatter))
plt.grid()
plt.savefig(path_+'/sueldos_quantiles_top_1_2023_UNAL.png',dpi = 300)
plt.show()

In [ ]:
import shutil
from google.colab import files

# Define the folder to be zipped
folder_to_zip = "resultadosUNAL"  # Replace with your folder name
zip_output_file = "resultadosUNAL.zip"  # Name of the output zip file

# Create a ZIP file
try:
    shutil.make_archive(zip_output_file.replace('.zip', ''), 'zip', folder_to_zip)
    print(f"Folder '{folder_to_zip}' has been successfully zipped as '{zip_output_file}'.")
except Exception as e:
    print(f"An error occurred while zipping the folder: {e}")

# Download the file
files.download(zip_output_file)